# 1. Чтение, нормализация, удаление стоп-слов

In [4]:
import gzip
import re

from dataclasses import dataclass
from typing import Iterator

from nltk.corpus import stopwords
from yargy.tokenizer import MorphTokenizer


@dataclass
class Text:
    label: str
    title: str
    content: str


def read_texts(fn: str) -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))


tokenizer = MorphTokenizer()
ru_stopwords = set(stopwords.words("russian"))

def normalize_text(text: str) -> str:
    # убираем знаки препинания
    words = re.findall(r'\b\w+\b', text.lower())
    # нормализация и убираем стоп-слова
    tokens = [
        tok.normalized for tok in tokenizer(' '.join(words))
        if (tok.normalized not in ru_stopwords) and (re.fullmatch(r'\b\w+\b', tok.normalized))
    ]
    return tokens

In [5]:
filepath = '../../data/news.txt.gz'
texts = list(read_texts(filepath))

corpus = [normalize_text(doc.content) for doc in texts]
labels = [doc.label for doc in texts]

In [6]:
texts[:5]

[Text(label='style', title='Rolex наградит победителей регаты', content='Парусная гонка Giraglia Rolex Cup пройдет в Средиземном море в 64-й раз. Победители соревнования, проводимого с 1953 года Yacht Club Italiano, помимо других призов традиционно получают в подарок часы от швейцарского бренда Rolex. Об этом сообщается в пресс-релизе, поступившем в редакцию «Ленты.ру» в среду, 8 мая. Rolex Yacht-Master 40 Фото: пресс-служба Mercury Соревнования будут проходить с 10 по 18 июня. Первый этап: ночной переход из Сан-Ремо в Сен-Тропе 10-11 июня (дистанция 50 морских миль — около 90 километров). Второй этап: серия прибрежных гонок в бухте Сен-Тропе с 11 по 14 июня. Финальный этап пройдет с 15 по 18 июня: оффшорная гонка по маршруту Сен-Тропе — Генуя (243 морских мили — 450 километров). Маршрут проходит через скалистый остров Джиралья к северу от Корсики и завершается в Генуе.Регата, с 1997 года проходящая при поддержке Rolex, считается одной из самых значительных яхтенных гонок в Средиземном

In [7]:
corpus[:5]

[['парусный',
  'гонка',
  'giraglia',
  'rolex',
  'cup',
  'пройти',
  'средиземный',
  'море',
  '64',
  'й',
  'победитель',
  'соревнование',
  'проводить',
  '1953',
  'год',
  'yacht',
  'club',
  'italiano',
  'помимо',
  'приз',
  'традиционно',
  'получать',
  'подарок',
  'часы',
  'швейцарский',
  'бренд',
  'rolex',
  'это',
  'сообщаться',
  'пресс',
  'релиз',
  'поступить',
  'редакция',
  'лента',
  'ру',
  'среда',
  '8',
  'май',
  'rolex',
  'yacht',
  'master',
  '40',
  'фото',
  'пресс',
  'служба',
  'mercury',
  'соревнование',
  'проходить',
  '10',
  '18',
  'июнь',
  'первый',
  'этап',
  'ночной',
  'переход',
  'сан',
  'рть',
  'сен',
  'тропа',
  '10',
  '11',
  'июнь',
  'дистанция',
  '50',
  'морской',
  'миля',
  'около',
  '90',
  'километр',
  'второй',
  'этап',
  'серия',
  'прибрежный',
  'гонка',
  'бухта',
  'сен',
  'тропа',
  '11',
  '14',
  'июнь',
  'финальный',
  'этап',
  'пройти',
  '15',
  '18',
  'июнь',
  'оффшорный',
  'гонка',
  'м

In [8]:
labels[:5]

['style', 'sport', 'media', 'economics', 'economics']

# 2. Векторное представление слов

Обучим Word2Vec и FastText

In [18]:
from gensim.models import Word2Vec, FastText

In [7]:
w2v = Word2Vec(sentences=corpus)
# сохраняем модель
w2v.save('w2v_default.model')

In [9]:
ft = FastText(sentences=corpus)
ft.save("fs_default.model")

In [10]:
w2v.wv.most_similar("новость")

[('итар', 0.6321862936019897),
 ('прайма', 0.5768018960952759),
 ('бёлтый', 0.5752525925636292),
 ('интерфакс', 0.5749506950378418),
 ('тасс', 0.5697027444839478),
 ('ренхап', 0.5616961717605591),
 ('service', 0.5563392639160156),
 ('агентство', 0.550072968006134),
 ('корреспондент', 0.5459086894989014),
 ('радиостанция', 0.539406955242157)]

In [11]:
ft.wv.most_similar("новость")

[('новостной', 0.8780274391174316),
 ('сонливость', 0.84711754322052),
 ('новосельцев', 0.8130514025688171),
 ('ведомость', 0.8108958601951599),
 ('цитируемость', 0.7947829365730286),
 ('справедливость', 0.7823488116264343),
 ('интерфакс', 0.7537603378295898),
 ('глупость', 0.7456846833229065),
 ('несправедливость', 0.7405885457992554),
 ('хвост', 0.7292425632476807)]

Обучим еще несколько моделей (будем варьировать параметры), чтобы в дальнейшем сравнить результаты для разных векторных представлений

In [12]:
# попробуем skip-gram, т.к. наш набор данных небольшой и будет полезно захватывать редкие слова
w2v_skip_gram = Word2Vec(sentences=corpus,
                         vector_size=70,
                         sg=1,
                         min_count=2)
w2v_skip_gram.save("w2v_skipgram.model")

In [14]:
# добавим иерархический softmax
w2v_hs = Word2Vec(sentences=corpus,
                  vector_size=70,
                  hs=1)
w2v_hs.save("w2v_hs.model")

# 3. Отобразим каждый документ в вектор

Загрузим модели

In [9]:
from gensim.models import Word2Vec, FastText

model_w2v = Word2Vec.load("w2v_default.model")
model_w2v_hs = Word2Vec.load("w2v_hs.model")
model_w2v_skipgram = Word2Vec.load("w2v_skipgram.model")
model_ft = FastText.load("fs_default.model")

С помощью обученных моделей векторного представления отобразим каждый документ в вектор, усреднив все вектора для слов документа. 

In [10]:
import numpy as np

def document_to_vector_avg(doc, model):
    tokens = [token for token in doc if token in model.wv.index_to_key]
    if len(tokens) == 0:
        return np.zeros(model.vector_size)
    else:
        return np.mean(model.wv[tokens], axis=0)

### Альтернатива: сумма

По сути, не должно быть сильных различий с средним, т.к. все равно является похожей линейной комбинацией

In [11]:
def document_to_vector_sum(doc, model):
    tokens = [token for token in doc if token in model.wv.index_to_key]
    if len(tokens) == 0:
        return np.zeros(model.vector_size)
    else:
        return np.sum(model.wv[tokens], axis=0)


### Альтернатива: медиана

Медиана устойчива к выбросам, хотя мы старались от них уйти еще в момент обучения моделей векторного представления слов. Однако медиана может дать интересный результат

In [ ]:
def document_to_vector_median(doc, model):
    tokens = [token for token in doc if token in model.wv.index_to_key]
    if len(tokens) == 0:
        return np.zeros(model.vector_size)
    else:
        return np.median(model.wv[tokens], axis=0)

### Альтернатива: взвешенное средение

Попробуем использовать дополнительную информацию о токенах, чтобы каждый word embedding [имел определенный вес в усреднении](https://www.kaggle.com/code/sahib12/document-embedding-techniques?scriptVersionId=31484359&cellId=44)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.2, min_df=10).fit([' '.join(text) for text in corpus])
feature_names = list(vectorizer.get_feature_names_out())
feature_names_dict = {word: idx for idx, word in enumerate(feature_names)}

In [13]:
tf_idf_matrix = vectorizer.transform([' '.join(text) for text in corpus])

def document_to_vector_weighted_avg(doc, idx_doc, model):
    tokens = [token for token in doc if token in model.wv.index_to_key
                                     and token in feature_names]
    doc_vector = np.sum([model.wv[token] * tf_idf_matrix[idx_doc, feature_names_dict[token]] for token in tokens],
                        axis=0)
    
    if len(tokens) == 0:
        return np.zeros(model.vector_size)
    else:
        return doc_vector / len(tokens)

# 4. Обучение и тестирование

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

models = [model_w2v, model_w2v_hs, model_w2v_skipgram, model_ft]

def train_test_process(model, doc_to_vec):
    X = np.array([doc_to_vec(text, model) for text in corpus])
    y = labels
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)
    
    # обучим SVM
    svm_classifier = SVC(kernel='linear', random_state=19)
    svm_classifier.fit(X_train, y_train)
    svm_predictions = svm_classifier.predict(X_test)
    print(f"Accuracy: {accuracy_score(y_test, svm_predictions):.4f}")
    print(f"F1: {f1_score(y_test, svm_predictions, average='weighted'):.4f}")

    # # обучим RF
    # rf_classifier = RandomForestClassifier(n_estimators=70, random_state=19)
    # rf_classifier.fit(X_train, y_train)
    # rf_predictions = rf_classifier.predict(X_test)
    # print(f"Accuracy: {accuracy_score(y_test, rf_predictions):.4f}")
    # print(f"F1: {f1_score(y_test, rf_predictions, average='weighted'):.4f}")

def train_test_process_advanced(model):
    X = np.array([document_to_vector_weighted_avg(corpus[i], i, model) for i in range(10_000)]) #len(corpus)
    y = labels
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)
    
    # обучим SVM
    svm_classifier = SVC(kernel='linear', random_state=19)
    svm_classifier.fit(X_train, y_train)
    svm_predictions = svm_classifier.predict(X_test)
    print(f"Accuracy: {accuracy_score(y_test, svm_predictions):.4f}")
    print(f"F1: {f1_score(y_test, svm_predictions, average='weighted'):.4f}")

    # # обучим RF
    # rf_classifier = RandomForestClassifier(n_estimators=70, random_state=19)
    # rf_classifier.fit(X_train, y_train)
    # rf_predictions = rf_classifier.predict(X_test)
    # print(f"Accuracy: {accuracy_score(y_test, rf_predictions):.4f}")
    # print(f"F1: {f1_score(y_test, rf_predictions, average='weighted'):.4f}")

### Подход с усреднением векторов слов

In [ ]:
for i, model in enumerate(models):
    print(f"{i+1} / {model}")
    train_test_process(model, document_to_vector_avg)
    print()

1 / Word2Vec<vocab=21225, vector_size=100, alpha=0.025>
Accuracy: 0.8110
F1: 0.8017

2 / Word2Vec<vocab=21225, vector_size=70, alpha=0.025>
Accuracy: 0.8360
F1: 0.8318

3 / Word2Vec<vocab=39927, vector_size=70, alpha=0.025>
Accuracy: 0.8265
F1: 0.8134

4 / FastText<vocab=21225, vector_size=100, alpha=0.025>
Accuracy: 0.7805
F1: 0.7674



Лучше всех себя в данном случае показала модель `Word2Vec` с применением иерархического softmax

### Альтернатива с суммой векторов

In [15]:
for i, model in enumerate(models):
    print(f"{i+1} / {model}")
    train_test_process(model, document_to_vector_sum)
    print()

1 / Word2Vec<vocab=21225, vector_size=100, alpha=0.025>
Accuracy: 0.8040
F1: 0.8037

2 / Word2Vec<vocab=21225, vector_size=70, alpha=0.025>
Accuracy: 0.8180
F1: 0.8161

3 / Word2Vec<vocab=39927, vector_size=70, alpha=0.025>
Accuracy: 0.8230
F1: 0.8222

4 / FastText<vocab=21225, vector_size=100, alpha=0.025>
Accuracy: 0.7970
F1: 0.7957



### Альтернатива с медианой

In [16]:
for i, model in enumerate(models):
    print(f"{i+1} / {model}")
    train_test_process(model, document_to_vector_median)
    print()

1 / Word2Vec<vocab=21225, vector_size=100, alpha=0.025>
Accuracy: 0.7985
F1: 0.7890

2 / Word2Vec<vocab=21225, vector_size=70, alpha=0.025>
Accuracy: 0.8150
F1: 0.8108

3 / Word2Vec<vocab=39927, vector_size=70, alpha=0.025>
Accuracy: 0.8150
F1: 0.8044

4 / FastText<vocab=21225, vector_size=100, alpha=0.025>
Accuracy: 0.7860
F1: 0.7795



### Альтернативный подход с взвешенным усреднением

In [17]:
for i, model in enumerate(models):
    print(f"{i+1} / {model}")
    train_test_process_advanced(model)
    print()

1 / Word2Vec<vocab=21225, vector_size=100, alpha=0.025>
Accuracy: 0.7580
F1: 0.7348

2 / Word2Vec<vocab=21225, vector_size=70, alpha=0.025>
Accuracy: 0.7945
F1: 0.7757

3 / Word2Vec<vocab=39927, vector_size=70, alpha=0.025>
Accuracy: 0.7710
F1: 0.7439

4 / FastText<vocab=21225, vector_size=100, alpha=0.025>
Accuracy: 0.7340
F1: 0.7104



# Выводы

* С помощью `gensim.FastText` и `gensim.Word2Vec` обучили 4 модели на предложенном корпусе текстов и получили векторное представление каждого слова в корпусе
* Для формирования векторного представления документов попробовали 4 метода: усреднение/сумма/медиана/взвешенное (с помощью tf-idf) усреднение word embeddings
* Лучшим сочетание оказалось **обучение Word2Vec (skip-gram) с применение иерархического softmax** и **отображение документа в вектор с помощью простого усреднения**